In [ ]:
# Startup databrokers and elastic search
from pprint import pprint

from rapidz.graph import _clean_text, readable_graph
from xpdan.vend.callbacks.core import Retrieve
from xpdan.vend.callbacks.zmq import Publisher
from xpdconf.conf import glbl_dict

from databroker_elasticsearch import load_elasticindex
from databroker_elasticsearch.brokersearch import BrokerSearch

from databroker import Broker
import yaml
dbs = {}
for yaml_file in ['raw', 'an']:
    with open(f'{yaml_file}.yml', 'r') as f:
        dbs[yaml_file] = Broker.from_config(yaml.load(f))
from databroker_elasticsearch.converters import register_converter


def filtertype(seq, tp):
    return (o for o in seq if isinstance(o, tp))


@register_converter
def tomo_usednodes(graph):
    nodes = []
    fmt = '{mod}.{name}'
    for n in graph['nodes']:
        st = n['stream']
        e = dict()
        e['ndtype'] = fmt.format(**st)
        ad = next(filtertype(st['args'], dict), None)
        al = next(filtertype(st['args'], list), None)
        if ad is not None:
            e['ndfunc'] = fmt.format(**ad)
        elif al is not None:
            e['ndfunc'] = next(filtertype(al, str))
        nodes.append(e)
    return nodes
        
an_db = dbs['an']
raw_db = dbs['raw']
raw_es = load_elasticindex('es-raw.yaml')
an_es = load_elasticindex('es-an.yaml')
raw_db_es = BrokerSearch(raw_db, raw_es)
an_db_es = BrokerSearch(an_db, an_es)

In [ ]:
# query raw es for dino tooth, print output
raw_es.qsearch()

In [ ]:
# query an_es/databroker for tomo recon
hdrs = an_db_es()
tomo_analysis_hdr = next(iter(hdrs))
print(tomo_analysis_hdr.stop)

In [ ]:
# replay analysis with no changes
from shed.replay import replay

# load the replay
graph, parents, data, vs = replay(raw_db, tomo_analysis_hdr)

# make the graph more accessible to humans by renaming things
# these names *should* match the names in the graph plot
for k, v in graph.nodes.items():
    v.update(label=_clean_text(str(v['stream'])).strip())
graph = readable_graph(graph)

# plot the graph
graph.nodes['data img FromEventStream']['stream'].visualize()

In [ ]:
# print the current reconstruction algorithm
print(graph.nodes['starmap; recon_wrapper']['stream'].kwargs)

# setup a publisher to send over to data viz
p = Publisher(glbl_dict['inbound_proxy_address'], prefix=b'tomo')
graph.nodes['img_tomo ToEventStream']['stream'].DBFriendly().starsink(p)

# rerun data processing
r = Retrieve(dbs['raw'].reg.handler_reg)
for v in vs:
    d = data[v['uid']]
    dd = r(*d)
    parents[v["node"]].update(dd)

In [ ]:
# replay with changes
# print the current reconstruction algorithm
print(graph.nodes['starmap; recon_wrapper']['stream'].kwargs)

# change to Algebreic
graph.nodes['starmap; recon_wrapper']['stream'].kwargs['algorithm'] = 'art'
print(graph.nodes['starmap; recon_wrapper']['stream'].kwargs)

# setup a publisher to send over to data viz
p = Publisher(glbl_dict['inbound_proxy_address'], prefix=b'tomo')
graph.nodes['img_tomo ToEventStream']['stream'].DBFriendly().starsink(p)

# rerun data processing
r = Retrieve(dbs['raw'].reg.handler_reg)
for v in vs:
    d = data[v['uid']]
    dd = r(*d)
    parents[v["node"]].update(dd)

In [ ]:
# Compare results

In [ ]:
# an_es for new data (via new recon algo)